In [12]:
import Data_Conversion
import Input_Parameters 
import Data_Conversion
import Passive_Model
import Solar_Generation
import Electrical_Load
import Baseline_CO

In [13]:
import numpy as np
import pandas as pd

In [ ]:
# Sensitivity Analysis on Loss of Load Cost, Linear Programming
# Fred Fan, Stanford University, 03/17/2025

In [14]:
# Define the base data directory, list of loss of load costs, and the weather year.
data_dir = "Data"

location = "HalfMoonBay"

# Example loss of load cost sensitivity
lol_costs = [10, 100, 1000] # $/kWh 

# 20 years of training data
weather_year_list_training = list(range(1998, 1999))  # use list(range(1998, 2018)) for full dataset
# 5 years of training data
weather_year_list_testing = list(range(2022, 2023))  # use list(range(2018, 2023)) for full dataset

# Set a random seed for consistency

total_cells = len(weather_year_list_training) + len(weather_year_list_testing) 

sequential_numbers = np.arange(1, total_cells + 1)

# Reshape the array into a 2D array with i rows and j columns
data = sequential_numbers.reshape(1, total_cells)

# Convert the array into a DataFrame
random_seeds = pd.DataFrame(data, columns=[f'Column_{k+1}' for k in range(total_cells)])

lat, lon, timezone = Data_Conversion.get_timezone_singlezone(data_dir, location)

print(random_seeds)

Reading file for HalfMoonBay: Data\NREL_NSRDB_HalfMoonBay\137344_37.49_-122.42_1998.csv
   Column_1  Column_2
0         1         2


In [15]:
# Create the nested dictionary
training_results = {lolc: {year: {} for year in weather_year_list_training} for lolc in lol_costs}

In [17]:
# Sensitivity Analysis on Loss of Load Cost

for j in range(len(weather_year_list_training)):
    
    year = weather_year_list_training[j]

    # Get a unique random seed number
    random_seed = random_seeds.iloc[0, j]
    print(random_seed)
    
    # Read NSRDB weather data of the given location of the given year
    NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

    # Prepare weather data file using NSRDB data
    weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lat, lon, timezone)

    # Prepare heating and cooling load using weather data and passive model
    NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lat)

    # Prepare solar PV capacity factor using weather data
    pv_cf = Solar_Generation.generate_pv(weather_data, lat)

    # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
    load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
    
    # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
    input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)
    
    for i in range(len(lol_costs)):
        
        lolc = lol_costs[i]
        
        # Run optimization function
        PV_Size, Battery_Size, PCM_Heating_Size, PCM_Cooling_Size, ObjValue, First_stage_cost, Second_stage_cost = Baseline_CO.Cap_Baseline_V1(input_df, lolc)

        # Store the variables in the nested dictionary
        training_results[lolc][year] = {
            'PV_Size': PV_Size,
            'Battery_Size': Battery_Size,
            'PCM_Heating_Size': PCM_Heating_Size,
            'PCM_Cooling_Size': PCM_Cooling_Size,
            'Total Cost': ObjValue,
            'Capital Cost': First_stage_cost,
            'Operation Cost': Second_stage_cost
        }



1
Reading file for HalfMoonBay: Data\NREL_NSRDB_HalfMoonBay\137344_37.49_-122.42_1998.csv
Read LP format model from file C:\Users\Fred\AppData\Local\Temp\tmp9e2ky3c1.pyomo.lp
Reading time = 0.38 seconds
x1: 157682 rows, 140165 columns, 424826 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 157682 rows, 140165 columns and 424826 nonzeros
Model fingerprint: 0x7f22c4a3
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-04, 2e+01]
Presolve removed 54062 rows and 38545 columns
Presolve time: 3.27s
Presolved: 103620 rows, 101620 columns, 430990 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.12s

In [18]:
print(training_results)

{10: {1998: {'PV_Size': 9.779, 'Battery_Size': 1.09, 'PCM_Heating_Size': 44.022, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20463.665, 'Capital Cost': 19941.154, 'Operation Cost': 522.511}}, 100: {1998: {'PV_Size': 9.848, 'Battery_Size': 1.628, 'PCM_Heating_Size': 69.417, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20670.493, 'Capital Cost': 20670.493, 'Operation Cost': 0.0}}, 1000: {1998: {'PV_Size': 9.848, 'Battery_Size': 1.628, 'PCM_Heating_Size': 69.417, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20670.493, 'Capital Cost': 20670.493, 'Operation Cost': 0.0}}}


In [19]:
# Initialize a dictionary to store averaged results
training_averaged_results = {}

# Iterate over each loss of load cost in the original dictionary (results)
for lolc, years_data in training_results.items():
    # Initialize a dictionary to store sums and counts for averaging
    sums = {
        'PV_Size': 0,
        'Battery_Size': 0,
        'PCM_Heating_Size': 0,
        'PCM_Cooling_Size': 0,
        'Total Cost': 0,
        'Capital Cost': 0,
        'Operation Cost': 0
    }
    count = 0

    # Iterate over each year for the loss of load cost
    for year, data in years_data.items():
        for key in sums:
            sums[key] += data[key]
        count += 1

    # Calculate averages and store in training_averaged_results
    training_averaged_results[lolc] = {key: value / count for key, value in sums.items()}

# Print the averaged results
print(training_averaged_results)

{10: {'PV_Size': 9.779, 'Battery_Size': 1.09, 'PCM_Heating_Size': 44.022, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20463.665, 'Capital Cost': 19941.154, 'Operation Cost': 522.511}, 100: {'PV_Size': 9.848, 'Battery_Size': 1.628, 'PCM_Heating_Size': 69.417, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20670.493, 'Capital Cost': 20670.493, 'Operation Cost': 0.0}, 1000: {'PV_Size': 9.848, 'Battery_Size': 1.628, 'PCM_Heating_Size': 69.417, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20670.493, 'Capital Cost': 20670.493, 'Operation Cost': 0.0}}


In [20]:
# Testing:

# Create the nested dictionary
testing_results = {lolc: {year: {} for year in weather_year_list_testing} for lolc in lol_costs}


for j in range(len(weather_year_list_testing)):
    
    year = weather_year_list_testing[j]
    
    # Get a unique random seed number
    random_seed = random_seeds.iloc[0, j+len(weather_year_list_training)]
    print(random_seed)
    
    # Read NSRDB weather data of the given location of the given year
    NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

    # Prepare weather data file using NSRDB data
    weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lat, lon, timezone)

    # Prepare heating and cooling load using weather data and passive model
    NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lat)

    # Prepare solar PV capacity factor using weather data
    pv_cf = Solar_Generation.generate_pv(weather_data, lat)

    # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
    load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
    
    # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
    input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)
    
    for i in range(len(lol_costs)):
        
        lolc = lol_costs[i]
        
        test_capacities = [training_averaged_results[lolc]['PV_Size'], training_averaged_results[lolc]['Battery_Size'], training_averaged_results[lolc]['PCM_Heating_Size'], training_averaged_results[lolc]['PCM_Cooling_Size']]
        
        ObjValue, First_stage_cost, Second_stage_cost = Baseline_CO.simulate(input_df, lolc, test_capacities)
        # Store the variables in the nested dictionary
        testing_results[lolc][year] = {
            'Total Cost': ObjValue,
            'Capital Cost': First_stage_cost,
            'Operation Cost': Second_stage_cost
        }

2
Reading file for HalfMoonBay: Data\NREL_NSRDB_HalfMoonBay\137344_37.49_-122.42_2022.csv
Read LP format model from file C:\Users\Fred\AppData\Local\Temp\tmpfwi51t82.pyomo.lp
Reading time = 0.37 seconds
x1: 157682 rows, 140160 columns, 385430 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 157682 rows, 140160 columns and 385430 nonzeros
Model fingerprint: 0xc3d1e358
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 4e+01]
Presolve removed 99180 rows and 56113 columns
Presolve time: 0.21s
Presolved: 58502 rows, 84047 columns, 187753 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.03s



In [21]:
# Initialize a dictionary to store averaged results
testing_averaged_results = {}

# Iterate over each loss of load cost
for lolc, years_data in testing_results.items():
    # Initialize a dictionary to store sums and counts for averaging
    sums = {
        'Total Cost': 0,
        'Capital Cost': 0,
        'Operation Cost': 0
    }
    count = 0

    # Iterate over each year for the loss of load cost
    for year, data in years_data.items():
        for key in sums:
            sums[key] += data[key]
        count += 1

    # Calculate averages
    testing_averaged_results[lolc] = {key: value / count for key, value in sums.items()}

# Print the averaged results
print(testing_averaged_results)

{10: {'Total Cost': 20191.301000000003, 'Capital Cost': 19941.185, 'Operation Cost': 250.116}, 100: {'Total Cost': 21709.704, 'Capital Cost': 20670.5, 'Operation Cost': 1039.204}, 1000: {'Total Cost': 31062.538, 'Capital Cost': 20670.5, 'Operation Cost': 10392.038}}


In [22]:
from pathlib import Path

# Define the output file path
output_file = "Results_SA_LoLC_LP.xlsx"

# Store Training Result
train_df = pd.DataFrame(training_averaged_results).T 
sheet_name_train = "Training Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"File '{output_file}' created with sheet '{sheet_name_train}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_train}'.")


# Store Testing Result
test_df = pd.DataFrame(testing_averaged_results).T 
sheet_name_test = "Testing Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"File '{output_file}' created with sheet '{sheet_name_test}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_test}'.")

File 'Results_SA_LoLC_LP.xlsx' created with sheet 'Training Results'.
Data written to 'Results_SA_LoLC_LP.xlsx' in sheet 'Testing Results'.
